# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where T
    lattice=T(a) * I(3)  # lattice is a cube of ``a`` Bohrs
    He = ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))
    atoms = [He => [[1/2; 1/2; 1/2]]]  # Helium at the center of the box

    model = model_DFT(lattice, atoms, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...), tol=tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770890628842         NaN   2.99e-01   0.80    9.0
  2   -2.772148985413   -1.26e-03   4.77e-02   0.80    1.0
  3   -2.772169891588   -2.09e-05   4.28e-03   0.80    2.0
  4   -2.772170722426   -8.31e-07   9.32e-05   0.80    2.0
┌ Warning: Negative ρ detected
│   min_ρ = -1.4514200747731162e-18
└ @ DFTK /home/runner/work/DFTK.jl/DFTK.jl/src/densities.jl:7
  5   -2.772170723004   -5.78e-10   1.00e-05   0.80    2.0
n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770822546728         NaN   2.99e-01   0.80    9.0
  2   -2.772062643376   -1.24e-03   4.78e-02   0.80    1.0
  3   -2.772082685891   -2.00e-05   4.12e-03   0.80    2.0
  4   -2.772083417463   -7.32e-07   1.01e-04   0.80    2.0
  5   -2.772083417804   -3.41e-10   1.31e-05   0.80    2.0


1.7736894455173748

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            Eₙ-Eₙ₋₁     ρout-ρin   α      Diag
---   ---------------   ---------   --------   ----   ----
  1   -2.770661117895         NaN   2.97e-01   0.80    8.0
  2   -2.772049156933   -1.39e-03   4.99e-02   0.80    1.0
  3   -2.772082652928   -3.35e-05   2.29e-03   0.80    1.0
  4   -2.772083416348   -7.63e-07   7.45e-05   0.80    2.0
  5   -2.772083417802   -1.45e-09   9.12e-06   0.80    2.0

Polarizability via ForwardDiff:       1.7725345223905822
Polarizability via finite difference: 1.7736894455173748
